### PyTorch Basic

In [1]:
# import modules
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
'''
 All of TorchVision datasets include 2 factors
 - transform: to transform sample data
 - target_transform: to transform target data
'''

# import Train Data from FashionMNIST
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# import Test Data from FashionMNIST
test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
# Set batch size
batch_size = 64

# Create DataLoader for train, test data
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# Check the output shape of DataLoader
for X, y in test_dataloader:
    print(f"Shape of X [N, Channel, Height, Width]: {X.shape}")
    print(f"Shape of y: {y.shape}")
    print(f"Data Type of y: {y.dtype}")
    break

Shape of X [N, Channel, Height, Width]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])
Data Type of y: torch.int64


In [5]:
# Before start training, check the device to train
device = ("cuda" if torch.cuda.is_available()
                else "mps" if torch.backends.mps.is_available()
                else "CPU")

print(f"Current Device: {device}")

Current Device: cuda


In [6]:
# Declare NeuralNetwork class based on nn.Module
class NeuralNetwork(nn.Module):
    # This will execute when the class object is called
    def __init__(self):
        super().__init__()  # Execute nn.Module
        self.flatten = nn.Flatten()

        # Neural Network Stack
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),  # Linear - The number of output features must be same as the number of input features on next Linear process
            nn.ReLU(),  # Activation Function
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 18)
        )
    # Declare how to send the data to the Neural Network
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits

# Pass the Neural Network computing to available device to accelerate the process
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()     # Loss function - Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)    # Optimizer - SGD (Stochastic Gradient Descent) with 0.001 learning rate

In [8]:
# Model Training Function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)     # Predict the train data with model
        loss = loss_fn(pred, y)     # Calculate loss of model prediction

        loss.backward()     # Back Propagation
        optimizer.step()
        optimizer.zero_grad()   # Initialize gradient before the next epoch

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()    # Evaluate the model

    test_loss, correct = 0, 0   # Initialize loss and accuracy

    # Initialize gradient
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)

            # Update loss and accuracy on every step
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(correct * 100):0.1f}% \n Average Loss: {test_loss:>8f} \n")

In [10]:
epochs = 50

# Train and evaluate the model
for t in range(epochs):
    print(f"Epoch {t+1} \n---------------------------------")

    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1 
---------------------------------
Loss: 2.904298 [   64 / 60000]
Loss: 2.857105 [ 6464 / 60000]
Loss: 2.803376 [12864 / 60000]
Loss: 2.770401 [19264 / 60000]
Loss: 2.722653 [25664 / 60000]
Loss: 2.637305 [32064 / 60000]
Loss: 2.625305 [38464 / 60000]
Loss: 2.527887 [44864 / 60000]
Loss: 2.487844 [51264 / 60000]
Loss: 2.394116 [57664 / 60000]
Test Error: 
 Accuracy: 28.1% 
 Average Loss: 2.373906 

Epoch 2 
---------------------------------
Loss: 2.396533 [   64 / 60000]
Loss: 2.374557 [ 6464 / 60000]
Loss: 2.241761 [12864 / 60000]
Loss: 2.252399 [19264 / 60000]
Loss: 2.186473 [25664 / 60000]
Loss: 2.054565 [32064 / 60000]
Loss: 2.108408 [38464 / 60000]
Loss: 1.985680 [44864 / 60000]
Loss: 1.981410 [51264 / 60000]
Loss: 1.865854 [57664 / 60000]
Test Error: 
 Accuracy: 61.0% 
 Average Loss: 1.871484 

Epoch 3 
---------------------------------
Loss: 1.928869 [   64 / 60000]
Loss: 1.885610 [ 6464 / 60000]
Loss: 1.725300 [12864 / 60000]
Loss: 1.760395 [19264 / 60000]
Loss: 1.65610

In [11]:
# Save the model
torch.save(model.state_dict(), "./model/torch_basic.pth")
print("Saved PyTorch Model State as ./model/torch_basic.pth")

Saved PyTorch Model State as ./model/torch_basic.pth


In [12]:
# Call the model state
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("./model/torch_basic.pth"))

C:\Users\khkim\AppData\Local\Temp\ipykernel_7160\2919288054.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./model/torch_basic.pth"))


<All keys matched successfully>

In [13]:
# Predict with loaded model
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
